In [82]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [39]:
adult_df = pd.read_csv('RDD_data/adult21.csv')
child_df = pd.read_csv('RDD_data/child21.csv')

adult_income_df = pd.read_csv('RDD_data/adultinc21.csv')
child_income_df = pd.read_csv('RDD_data/childinc21.csv')



In [ ]:
#RATCAT_A 

#Merge income with Adult and Child dataframes based on 'HHX' 

adult_merged = pd.merge(adult_df, adult_income_df, on = 'HHX')
child_merged = pd.merge(child_df, child_income_df, on = 'HHX')

In [51]:
child_df

,URBRRL,RATCAT_C,IMPINCFLG_C,SHTCVD19AV_C,SHTCVD19NM_C,PPSU,PSTRAT,HISPALLP_C,RACEALLP_C,SCHDYMSSTC_C,...,PHSTAT_C,HHSTAT_C,INTV_MON,RECTYPE,IMPNUM_C,RELCHPARENTP1_C,RELCHPARENTP2_C,WTFA_C,HHX,POVRATTC_C
0,1,14,0,NaN,NaN,3,115,3,2,4.0,...,1,1,1,20,1,1,4,6128.031,H025636,5.13
1,1,7,0,NaN,NaN,3,115,3,2,0.0,...,2,1,1,20,1,1,4,4529.750,H018455,1.96
2,1,14,0,NaN,NaN,1,115,2,1,0.0,...,1,1,1,20,1,4,4,5790.022,H047222,7.20
3,1,13,0,NaN,NaN,1,115,2,1,10.0,...,1,1,1,20,1,1,1,16068.950,H020084,4.85
4,3,4,0,NaN,NaN,1,103,3,2,NaN,...,3,1,1,20,1,3,4,4668.747,H046058,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8256,4,9,0,NaN,NaN,27,146,2,1,NaN,...,1,1,12,20,1,1,1,6346.783,H007394,2.59
8257,4,5,0,NaN,NaN,29,146,2,1,3.0,...,1,1,12,20,1,1,4,20683.243,H038392,1.44
8258,3,12,0,NaN,NaN,114,108,2,1,0.0,...,2,1,12,20,1,1,1,2153.966,H012473,4.19
8259,3,8,0,NaN,NaN,114,108,2,1,NaN,...,1,1,12,20,1,1,1,3875.047,H002032,2.26


In [44]:
adult_income_df

,RATCAT_A,IMPNUM_A,IMPINCFLG_A,RECTYPE,POVRATTC_A,HHX
0,7,1,0,30,1.93,H056808
1,7,2,0,30,1.93,H056808
2,7,3,0,30,1.93,H056808
3,7,4,0,30,1.93,H056808
4,7,5,0,30,1.93,H056808
...,...,...,...,...,...,...
294815,13,6,0,30,4.89,H052354
294816,13,7,0,30,4.89,H052354
294817,13,8,0,30,4.89,H052354
294818,13,9,0,30,4.89,H052354


In [80]:
#Get rows we want


#HISTOP_COST_A is Cost Increase

#HISTOPJOB_A is Number of months without coverage

#PAYBLL12M_A is problems paying medical bills, past 12 months

#MEDDL12M_A is Delayed medical care due to cost, past 12m

#MEDNG12M_A is Needed medical care but did not get it due to cost, past 12m

adult_data = adult_df[['HHX', 'AGEP_A', 'SEX_A', 'HOSPONGT_A', 'RATCAT_A',
                 'CHIP_A','PAYBLL12M_A',
                  'MEDDL12M_A','MEDNG12M_A']]
child_data = child_df[['HHX', 'AGEP_C', 'SEX_C', 'HOSPONGT_C', 'RATCAT_C',
                 'CHIP_C', 'PAYBLL12M_C',
                  'MEDDL12M_C','MEDNG12M_C']]


child_data = child_data.loc[(child_data['AGEP_C'] >= 14)]
#df['new_column'] = df['a'].apply(lambda x: 1 if x == 0 or x == 1 else 0)
child_data['Treatment'] = child_df['CHIP_C'].apply(lambda x: 1 if x == 1 or x == 2 else 0)

adult_data = adult_data.loc[adult_data['AGEP_A'] <= 24] #Experimental group should not be much higher than 23. 



#Rename Columns
adult_data.rename(columns = {'AGEP_A':'Age', 'SEX_A':'Sex', 'HOSPONGT_A': 'Hospitalized', 
                         'RATCAT_A': 'Income_Ratio', 'CHIP_A': 'CHIP',
                          'PAYBLL12M_A': 'PayBill', 'MEDDL12M_A': 'DelayedCareCost',
                             'MEDNG12M_A':'NoCareCost'},inplace=True)

child_data.rename(columns = {'AGEP_C':'Age', 'SEX_C':'Sex', 'HOSPONGT_C': 'Hospitalized', 
                         'RATCAT_C': 'Income_Ratio', 'CHIP_C': 'CHIP',
                          'PAYBLL12M_C': 'PayBill', 'MEDDL12M_C': 'DelayedCareCost',
                          'MEDNG12M_C':'NoCareCost'}, inplace=True)

adult_data['Treatment'] = 0

In [81]:
child_data

,HHX,Age,Sex,Hospitalized,Income_Ratio,CHIP,PayBill,DelayedCareCost,NoCareCost,Treatment
0,H025636,17,1,2.0,14,1,2,2,2,1
5,H019593,16,1,2.0,5,1,1,2,2,1
7,H050208,14,1,2.0,14,3,2,2,2,0
11,H066576,17,1,2.0,11,3,2,2,2,0
12,H019151,16,2,2.0,12,3,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...
8212,H062351,17,1,2.0,7,3,2,2,2,0
8217,H009328,15,2,2.0,13,3,2,2,2,0
8218,H049980,15,2,2.0,8,3,2,2,2,0
8229,H064558,17,2,2.0,10,3,2,2,2,0


In [103]:
#RUN RDD ON above target variable to be 'PAYBLL12M_C'
rdd_data = pd.concat([child_data, adult_data])

#Choose relevant covariates


#Run Regression on Treatment effect for pre-intervention

rdd_x = rdd_data[['Treatment', 'Age', 'Income_Ratio']]
rdd_y = rdd_data['NoCareCost']

response_variables = ['NoCareCost', 'DelayedCareCost', 'Hospitalized', 'PayBill']

for response in response_variables:
    rdd_y = rdd_data[response]
    regression = sm.OLS(rdd_y, rdd_x)
    fit = regression.fit()
    print(fit.pvalues)
    print("Effect of response" + str(response) + " is " + str(fit.params['Treatment']) 
          + " with p_value " + str(fit.pvalues['Treatment']))

#For Hospitalized, we get a causal effect of 0.13 

Treatment       2.970677e-18
Age             0.000000e+00
Income_Ratio    3.901486e-56
dtype: float64
Effect of responseNoCareCost is 0.5629181471624296 with p_value 2.9706772209351832e-18
Treatment       4.598329e-17
Age             0.000000e+00
Income_Ratio    4.996996e-55
dtype: float64
Effect of responseDelayedCareCost is 0.5561329990381586 with p_value 4.5983289166548155e-17
Treatment       5.870980e-15
Age             0.000000e+00
Income_Ratio    4.648720e-54
dtype: float64
Effect of responseHospitalized is 0.5095780648647121 with p_value 5.870979715923213e-15
Treatment       4.137333e-06
Age             0.000000e+00
Income_Ratio    1.461434e-50
dtype: float64
Effect of responsePayBill is 0.3739463257831849 with p_value 4.1373328126744205e-06
